# Projet 3 : développer à partir de l’expression de gènes (puce à ADN) un outil de diagnostic, capable de prédire plusieurs types de cancer.

Etudiants :
- Mattei Simon
- Mercado Pablo

## Introduction
### Un rappel sur ce qu'est le cancer
Le cancer est la prolifération de cellules anormales dans un tissu. Elles peuvent utiliser le système sanguin pour voyager et aller affecter d'autres tissus. Certains cancers sont d'origine génétique et ils existent plusieurs facteurs de risques (consommation excessive de tabac et alcool, exposition au soleil, obésité...). Manger une [pomme](https://pubmed.ncbi.nlm.nih.gov/23091184/), faites de l'exercice et manger sainement pour éviter le cancer!

Bien que l'âge ne soit pas la cause directe du cancer, on observe un taux d'incidence plus élevée chez les personnes agées. Avec le vieillissement de la population, la recherche du cancer est un domaine qui pourrait sauver beaucoup de vies.

### Types de cancer
On peut classer les [cancers](https://fr.wikipedia.org/wiki/Cancer) par les organes (et tissus) qu'ils touchent :
*    le cancer du sein
*    le cancer du côlon
*    le cancer du pancréas
*    le cancer de la prostate
*    le cancer du cerveau
*    le myélome multiple : cancer de la moelle osseuse
*    les leucémies
*    le sarcome de Kaposi : cancer des vaisseaux sanguins
*    les lymphomes
*    le cancer des testicules
*    le cancer des poumons
*    le cancer des reins
*    le cancer du foie

### Panel NCI60
Le dataset utilisé prend des [lignées cellulaires (cell line)](https://fr.wikipedia.org/wiki/Lign%C3%A9e_cellulaire_(m%C3%A9decine)), c-à-d des cellules pouvant théoriquement se diviser à l'infini.

In [1]:
import numpy as np

In [2]:
from Bio import Geo

def load_geo(myfile):
    handle = open(myfile)
    records = Geo.parse(handle)
    return records

records = load_geo('GDS4296.soft')

On extrait quelques informations sur le panel...

In [3]:
f = open("soft-info.txt", "w")

while True:
    try:
        nr = next(records)
        f.write(str(nr))
        f.write('\n---------------------- ENTITY_ATTRIBUTES:------------------------------\n')
        f.write(str(nr.entity_attributes))
    except StopIteration:
        print("done reading")
        break

done reading


Extrayons les données utiles: niveaux d'expressions des conditions et sources des conditions.

In [4]:
records = load_geo('GDS4296.soft')

# description of each condition (cell type)
id_cond_tissue = {}

diseases = set()
#disease_with_details = set()

# loop over records
for r in records:
    rea = r.entity_attributes

    # condition for cell type/tissue/disease state
    if 'subset_dataset_id' in rea:
        sample_list = rea['subset_sample_id'].split(',')
        desc = rea['subset_description']

        if rea['subset_type'] == 'tissue':
            for sample in sample_list:
                id_cond_tissue[sample] = desc
                diseases.add(desc)

        # si l'on veut plus de détail sur la variété de cancer
        # e.g. https://fr.wikipedia.org/wiki/Astrocytome
        #if rea['subset_type'] == 'disease state':
            #for sample in sample_list:
                #id_cond_disease[sample] = desc
                #disease_with_details.add(desc)

    if 'dataset_table_begin' in rea:
        data = r.table_rows

# converts data to an array (more practical to handle)
data = np.array(data)
print(len(id_cond_tissue))

174


On désire connaître la proportion de cancer répertorié dans les échantillons. Histoire de vérifier qu'il y a suffisamment d'échantillons pour faire notre classificateur.

In [5]:
cancer_sample_count = {}
for cancer in diseases:
    cancer_sample_count[cancer] = 0
    for sample in id_cond_tissue:
        if cancer == id_cond_tissue[sample]:
            cancer_sample_count[cancer] = cancer_sample_count[cancer] + 1

total_sample = len(id_cond_tissue)

print("{: >20} {: >20}".format("Cancer location", '# échantillons'), '\tquantité (%)')
print('------------------------------------------------------------------')
for cancer in cancer_sample_count:
    print("{: >20} {: >20}".format(cancer, cancer_sample_count[cancer]), '\t', "{0:.2f}".format(cancer_sample_count[cancer]/total_sample*100))

     Cancer location       # échantillons 	quantité (%)
------------------------------------------------------------------
               colon                   21 	 12.07
            leukemia                   18 	 10.34
            prostate                    6 	 3.45
               renal                   23 	 13.22
                 CNS                   18 	 10.34
 non-small cell lung                   26 	 14.94
             ovarian                   21 	 12.07
            melanoma                   26 	 14.94
              breast                   15 	 8.62


Seul les échantillons de la prostate sont en petites quantités (3% de l'échantillonage).

Afin de mener nos recherches en étant objectif, on divise notre jeu de données en données de train et données de test:

In [6]:
import random

# init dictionnary with disease - empty list
diseases_cond = {}
for disease in diseases:
    diseases_cond[disease] = []

# add every condition to dictionnary
for cond in id_cond_tissue:
    diseases_cond[id_cond_tissue[cond]].append(cond)

# shuffle conditions and add them to train/test conditions
test_conds = []
train_conds = []
for disease in diseases_cond:
    random.shuffle(diseases_cond[disease])
    idx_sep = int(len(diseases_cond[disease]) / 5) # 80/20%
    test_conds += diseases_cond[disease][:idx_sep]
    train_conds += diseases_cond[disease][idx_sep:]

# seperate data according to conditions
idx_test = []
for cond in test_conds:
    idx_test.append(data[0,:].tolist().index(cond))

idx_train = list(range(0,len(id_cond_tissue)+2))
[idx_train.remove(i) for i in idx_test]

data_test = data[:,idx_test]
data_train = data[:,idx_train]

print(np.shape(data_train))
print(np.shape(data_test))

(54676, 144)
(54676, 32)


On veut sélectionner les gènes pertinents dans notre jeu de train pour former notre classificateur. Affichons déjà les niveaux d'expression pour un gène aux travers des échantillons:

In [ ]:
unique_colors = ['c','g','r','b','k']
colors_disease_type = []

# for every type in cell_type, find the corresponding colums in data and add them to idx_cell_type
for j,ct in enumerate(cell_type):
    elements_cell_type = [ i+2 for i,c in enumerate(data[0,2:]) if id_cond[c]==ct]
    idx_cell_type.extend(elements_cell_type)
    colors_cell_type.extend(unique_colors[j]*len(elements_cell_type))
print('\nidx_cell_type:\t'+str(idx_cell_type))
print('\ncolors_cell_type:\t'+str(colors_cell_type))

# re-order columns of data (= conditions) with idx_cell_type
data = data[:,idx_cell_type]

In [7]:
def plot_gene_expression_across_conditions(data, gene, cancer_type, colors_cancer_type):
    print()

gene = '1007_s_at'

plot_gene_expression_across_conditions(test_data, gene, diseases, )

NameError: name 'test_data' is not defined